## KomaMRI im Optimized Shaped Pulse

In [1]:
using KomaMRI, MAT, Plots, WebIO

### Compartments
- Sample A: T1 = 100ms, T2 = 50ms 
- Sample B: T1 = 50ms, T2 = 25ms

### Scanner

- B0 =     1.5 [T] - main magnetic field
- B1 =    1.0e-5 [T] - maximum RF amplitude
- Gmax =   0.06 [T/m] - maximum gradient
- Smax =   500 [mT/m/ms] - maximum slew-rate
- ADC_Δt = 2.0e-6 [s] - ADC raster time
- seq_Δt = 1.0e-5 [s] - sequence-block raster time
- GR_Δt =  1.0e-5 [s] - gradient raster time
- RF_Δt =  1.0e-6 [s] - RF raster time
- RF_ring_down_T = 2.0e-5 [s] - RF ring down time
- RF_dead_time_T = 0.0001 [s] - RF dead time
- ADC_dead_time_T = 1.0e-5 [s] - ADC dead time

### MATLAB data

In [2]:
function read_data(file_path)
    # MATLAB data
    mat_data = matread(file_path)
    RF_Hz = mat_data["b1"]
    tf_sp = mat_data["tf_s"]
    t_sp = mat_data["t_s"]
    Mmax = mat_data["Mmax"]
    Mmin = mat_data["Mmin"]

    return RF_Hz, tf_sp, t_sp, Mmax, Mmin
end

read_data (generic function with 1 method)

### Functions

#### Sequence

In [3]:
function create_sequence(RF_T, tf_sp, nADC, durADC)
    # RF block
    exc = RF(RF_T', tf_sp)
    
    # ADC block
    aqc = ADC(nADC, durADC)
    
    # Concatenating
    seq = Sequence()
    seq += exc
    seq += aqc
    
    return seq
end

create_sequence (generic function with 1 method)

#### Signal

In [4]:
function simulate_signal(obj, seq, sys)
    signal = simulate(obj, seq, sys; simParams=Dict("return_type" => "state"))
    Mx = real(signal.xy)[:]
    My = imag(signal.xy)[:]
    Mz = signal.z[:]

    return Mx, My, Mz
end

simulate_signal (generic function with 1 method)

#### Fidelity

In [5]:
function calculate_fidelity(Mx, My, Mz, Mref)
    fidelity_Mx = round(abs(Mx[] - Mref[2, end])*100, digits=2)
    fidelity_My = round(abs(My[] - Mref[3, end])*100, digits=2)
    fidelity_Mz = round(abs(Mz[] - Mref[4, end])*100, digits=2)

    return fidelity_Mx, fidelity_My, fidelity_Mz
end

calculate_fidelity (generic function with 1 method)

#### Magnetization Dynamics

In [6]:
function simulate_magnetization_dynamics(RF_T, t_sp, sys, obj1, nADC, durADC)
    pieces = length(RF_T)
    M_koma = zeros(Float64, 3, pieces)
    t_koma = zeros(Float64, 1, pieces)
    
    for i in 1:pieces
        blocks = Int(length(RF_T) / pieces)
        rf_block = RF_T[1, 1:i*blocks]'
        t_block = t_sp[1, i*blocks]

        seq1 = create_sequence(rf_block, t_block, nADC, durADC)

        Mx, My, Mz = simulate_signal(obj1, seq1, sys)

        M_koma[1, i] = Mx[]
        M_koma[2, i] = My[]
        M_koma[3, i] = Mz[]
        t_koma[1, i] = t_block
    end
    
    return M_koma[2, :], M_koma[3, :], t_koma'
end

simulate_magnetization_dynamics (generic function with 1 method)

#### Plots

In [7]:
function plot_results(t_evol, Mmax, t_koma, Mz_koma)
    plotly()
    plot(t_evol, Mmax[4, :], line=:solid, marker=:circle, label="OC Grape")
    plot!(t_koma, Mz_koma, seriestype=:line, marker=:circle, label="KomaMRI")
end

plot_results (generic function with 1 method)

### Data for simulations

#### Load MATLAB data

In [8]:
oc_path = "/Users/amandanicotina/Documents/Julia/Projects/KomaMRIScripts/OC_Grape/OC_fields/oc_field.mat"
RF_Hz, tf_sp, t_sp, Mmax, Mmin = read_data(oc_path)

([-0.004595534572678076 -0.005070846407546822 … 4.03368665287538 -1.69328114318714], 1.056, [0.0 0.1056 … 0.8448 0.9504], [1.0 1.0 … 1.0 1.0; 0.0 0.0 … 0.0 0.0; 0.0 -0.0024604335412838738 … 0.8033397195397103 -0.028231915256496723; 1.0 0.9999964585200963 … 0.16085434326647235 0.7900076913059519], [1.0 1.0 … 1.0 1.0; 0.0 0.0 … 0.0 0.0; 0.0 -0.002014872998520531 … 0.7196685805165768 -0.1659950028094887; 1.0 0.9999972726826462 … 0.14310148598291272 0.7643335838722631])

#### Scanner

In [9]:
sys = Scanner()

Scanner
  B0: Float64 1.5
  B1: Float64 1.0e-5
  Gmax: Float64 0.06
  Smax: Int64 500
  ADC_Δt: Float64 2.0e-6
  seq_Δt: Float64 1.0e-5
  GR_Δt: Float64 1.0e-5
  RF_Δt: Float64 1.0e-6
  RF_ring_down_T: Float64 2.0e-5
  RF_dead_time_T: Float64 0.0001
  ADC_dead_time_T: Float64 1.0e-5


#### Phantom

In [10]:
obj = 
Phantom{Float64}(name = "spin1", x = [0.], T1 = [274e-3], T2 = [237e-3], Δw = [2π*7450]);
Mref = Mmax;

#### Create sequence

In [11]:
# convert to Tesla
RF_T = RF_Hz/γ;

# 1st block -> RF block
exc = RF(RF_T', tf_sp);

# 2nd block -> ADC block
nADC = 1 ;
durADC = 1e-3 ;

seq =  create_sequence(RF_T, tf_sp, nADC, durADC);

# Plot Sequence
# p1 = plot_seq(seq; slider = false, height = 300, max_rf_samples=Inf)

Sequence[ τ = 1057.0 ms | blocks: 3 | ADC: 1 | GR: 0 | RF: 1 | DEF: 0 ]

#### Calculate signal and fidelity

In [12]:
(Mx, My, Mz) = simulate_signal(obj, seq, sys)
(Fx, Fy, Fz) = calculate_fidelity(Mx, My, Mz, Mref)

┌ Info: The GPU function is being called but the GPU is not accessible. 
│ Defaulting back to the CPU. (No action is required if you want to run on the CPU).
└ @ KomaMRI /Users/amandanicotina/.julia/packages/KomaMRI/qZTLv/src/simulation/GPUFunctions.jl:29
┌ Info: Running simulation in the CPU with 1 thread(s)
│   koma_version = 0.7.3
│   sim_method = KomaMRI.Bloch()
│   spins = 1
│   time_points = 21122
│   adc_points = 1
└ @ KomaMRI /Users/amandanicotina/.julia/packages/KomaMRI/qZTLv/src/simulation/SimulatorCore.jl:219


Progress:  10%|███▉                                     |  ETA: 0:00:39
  simulated_blocks:  2
  rf_blocks:         2





Progress:  52%|█████████████████████▌                   |  ETA: 0:00:05
  simulated_blocks:  11
  rf_blocks:         11





Progress:  86%|███████████████████████████████████▏     |  ETA: 0:00:01
  simulated_blocks:  18
  rf_blocks:         18





Progress:  95%|███████████████████████████████████████  |  ETA: 0:00:00
  simulated_blocks:  20
  rf_blocks:         20





Progress: 100%|█████████████████████████████████████████| Time: 0:00:07
  simulated_blocks:  21
  rf_blocks:         20
  acq_samples:       1


  7.954935 seconds (31.47 M allocations: 1.501 GiB, 3.42% gc time, 85.34% compilation time: 1% of which was recompilation)


(0.05, 2.75, 20.99)

#### Simulate magnetization dynamics

In [13]:
(Mykoma, Mzkoma, tkoma) = 
simulate_magnetization_dynamics(RF_T, t_sp, sys, obj, nADC, durADC)

  0.000684 seconds (224 allocations: 12.516 KiB)


┌ Info: Running simulation in the CPU with 1 thread(s)
│   koma_version = 0.7.3
│   sim_method = KomaMRI.Bloch()
│   spins = 1
│   time_points = 2
│   adc_points = 1
└ @ KomaMRI /Users/amandanicotina/.julia/packages/KomaMRI/qZTLv/src/simulation/SimulatorCore.jl:219
┌ Info: Running simulation in the CPU with 1 thread(s)
│   koma_version = 0.7.3
│   sim_method = KomaMRI.Bloch()
│   spins = 1
│   time_points = 2114
│   adc_points = 1
└ @ KomaMRI /Users/amandanicotina/.julia/packages/KomaMRI/qZTLv/src/simulation/SimulatorCore.jl:219


Progress:  81%|█████████████████████████████████▎       |  ETA: 0:00:00
  simulated_blocks:  17
  rf_blocks:         17





Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
  simulated_blocks:  21
  rf_blocks:         20
  acq_samples:       1


  0.120524 seconds (413.06 k allocations: 19.881 MiB)


...




Progress:  86%|███████████████████████████████████▏     |  ETA: 0:00:00
  simulated_blocks:  18
  rf_blocks:         18





Progress:  95%|███████████████████████████████████████  |  ETA: 0:00:00
  simulated_blocks:  20
  rf_blocks:         20





Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
  simulated_blocks:  21
  rf_blocks:         20
  acq_samples:       1


  1.069418 seconds (3.95 M allocations: 181.593 MiB, 2.59% gc time)


([0.0, -7.945185416247114e-7, -4.542959004538716e-7, 4.070092472829856e-6, 9.953600965673104e-8, -2.0317829694249667e-5, 8.766131941229105e-6, 5.9752012020908296e-5, -0.00028658253722824156, -0.00018522364553064108], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0000758171081543, 0.9998964667320251, 0.9999470114707947], [0.0; 0.1056; … ; 0.8448; 0.9504;;])

#### Plots

In [20]:
t_evol = LinRange(0.0, tf_sp, Int(length(RF_T)+1))
plot_results(t_evol, Mmax, tkoma, Mzkoma)
